In [ ]:
from utils import CNFFormula
from main import solve_maxsat
import os

best_v_list = []
best_num_satisfied_list = []
# test all files in ../wcnfdata
for file in os.listdir("./wcnfdata"):
    print(file)
    with open(os.path.join("./wcnfdata", file), 'r') as f:
        formula_str = f.read()
    
    formula = CNFFormula.from_dimacs(formula_str)
    best_v, best_num_satisfied = solve_maxsat(
        formula, 
        max_time=300, 
        heuristic_interval=100, 
        batch_size=1024)

    best_v_list.append(best_v)
    best_num_satisfied_list.append(best_num_satisfied)

    print(f"{file}, {best_num_satisfied}")

brock200_3.clq.wcnf
Step 100, Best: 813, current: 810, formula size: 1038, Elapsed Time: 16.84s
Step 200, Best: 818, current: 803, formula size: 1038, Elapsed Time: 29.70s
Step 300, Best: 818, current: 801, formula size: 1038, Elapsed Time: 44.76s
Step 400, Best: 818, current: 801, formula size: 1038, Elapsed Time: 60.85s
Step 500, Best: 818, current: 806, formula size: 1038, Elapsed Time: 77.98s
Step 600, Best: 818, current: 802, formula size: 1038, Elapsed Time: 97.43s
Step 700, Best: 818, current: 805, formula size: 1038, Elapsed Time: 110.33s
Step 800, Best: 818, current: 802, formula size: 1038, Elapsed Time: 123.68s
Step 900, Best: 818, current: 806, formula size: 1038, Elapsed Time: 136.62s
Step 1000, Best: 818, current: 803, formula size: 1038, Elapsed Time: 149.38s
Step 1100, Best: 818, current: 808, formula size: 1038, Elapsed Time: 162.05s
Step 1200, Best: 818, current: 796, formula size: 1038, Elapsed Time: 174.84s
Step 1300, Best: 818, current: 804, formula size: 1038, Ela

RuntimeError: einsum(): subscript h has size 7 for operand 1 which does not broadcast with previously seen size 8

In [ ]:
import torch
from utils import CNFFormula
from main import count_satisfied_clauses
v = torch.tensor([[0,1,1,0,0,0,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,1,0,1,1,0,0,1,0,0,1,0,1,1,0,1,1,0,0,1]])

with open('./wcnfdata/brock200_3.clq.wcnf', 'r') as f:
    formula_str = f.read()
formula = CNFFormula.from_dimacs(formula_str)
best_v, best_num_satisfied = count_satisfied_clauses(
    formula, 
    v
)
print(best_num_satisfied)

822


In [4]:
v_str = "111111101111111001111111111111010111111111100011011100111011101101110001111111101111011011111111011110011111111111101110110110011110010"

v = torch.tensor([[int(i) for i in v_str]])

with open('./wcnfdata/data.135.wcnf', 'r') as f:
    formula_str = f.read()
formula = CNFFormula.from_dimacs(formula_str)
best_v, best_num_satisfied = count_satisfied_clauses(
    formula, 
    v
)
print(best_num_satisfied)

3046


In [5]:
from models.rbm import clauseRBM
import torch

rbm_random = clauseRBM(7, F_s=-1.0, num_epochs=0, lr=0.01, device='cpu', verbose=True)
rbm_trained = clauseRBM(7, F_s=-1.0, num_epochs=10000, lr=0.01, device='cpu', verbose=True)

v_neg = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
v_pos1 = torch.tensor([[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
v_pos2 = torch.tensor([[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]])
v_pos3 = torch.tensor([[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]])
v_pos4 = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]])
v_pos5 = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]])

print(rbm_random.free_energy(v_neg).item(), rbm_random.free_energy(v_pos1).item(), rbm_random.free_energy(v_pos2).item(), rbm_random.free_energy(v_pos3).item(), rbm_random.free_energy(v_pos4).item())
print(rbm_trained.free_energy(v_neg).item(), rbm_trained.free_energy(v_pos1).item(), rbm_trained.free_energy(v_pos2).item(), rbm_trained.free_energy(v_pos3).item(), rbm_trained.free_energy(v_pos4).item())

Saved pre-trained RBM to rbm_models\rbm_length_7_F_-1.0_num_epochs_0_lr_0.01_device_cpu.pth
Epoch 200, Loss: 0.5294787287712097
Epoch 400, Loss: 0.18371400237083435
Epoch 600, Loss: 0.08564762771129608
Epoch 800, Loss: 0.05263989791274071
Epoch 1000, Loss: 0.03903345391154289
Epoch 1200, Loss: 0.0323796309530735
Epoch 1400, Loss: 0.028537021949887276
Epoch 1600, Loss: 0.025904139503836632
Epoch 1800, Loss: 0.023803526535630226
Epoch 2000, Loss: 0.021942131221294403
Epoch 2200, Loss: 0.02020268887281418
Epoch 2400, Loss: 0.01855131797492504
Epoch 2600, Loss: 0.01699143461883068
Epoch 2800, Loss: 0.015539235435426235
Epoch 3000, Loss: 0.014211130328476429
Epoch 3200, Loss: 0.013018311001360416
Epoch 3400, Loss: 0.01196541078388691
Epoch 3600, Loss: 0.011051185429096222
Epoch 3800, Loss: 0.010269776917994022
Epoch 4000, Loss: 0.00961202010512352
Epoch 4200, Loss: 0.0090664466843009
Epoch 4400, Loss: 0.008620152249932289
Epoch 4600, Loss: 0.00825975276529789
Epoch 4800, Loss: 0.00797247048

In [4]:
from utils import CNFFormula, count_satisfied_clauses
from models.rbm import formulaRBM
import torch

with open('./wcnfdata/brock200_3.clq.wcnf', 'r') as f:
    formula_str = f.read()
formula = CNFFormula.from_dimacs(formula_str)
n_visible = formula.num_vars
v_init = torch.bernoulli(torch.full((1, n_visible), 0.5, device="cuda"))

print(count_satisfied_clauses(formula, v_init.cpu()))

rbm_trained = formulaRBM(formula,
                         num_epochs=10000,
                         device="cuda")

v = v_init.clone()
for _ in range(100):
    h_sample, _ = rbm_trained.sample_h_given_v(v)
    v_sample, p_v_given_h = rbm_trained.sample_v_given_h(h_sample)
    v = v_sample

    # plot the F_s each step

print(f"trained {count_satisfied_clauses(formula, v.cpu())}")
v1 = v.clone()


rbm_random = formulaRBM(formula,
                        num_epochs=0,
                        device="cuda")

v = v_init.clone()
for _ in range(100):
    h_sample, _ = rbm_random.sample_h_given_v(v)
    v_sample, p_v_given_h = rbm_random.sample_v_given_h(h_sample)
    v = v_sample
print(f"random {count_satisfied_clauses(formula, v.cpu())}")

v2 = v.clone()


(tensor([[1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
         0., 1., 1., 0.]]), 783)
trained (tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
         0., 1., 1., 0.]]), 755)
random (tensor([[0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
         0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
         0., 0., 0., 1.]]), 789)


In [2]:
print(rbm_trained.free_energy(v_init))
print(rbm_trained.free_energy(v1))
print(rbm_trained.free_energy(v2))

tensor([819.0408], device='cuda:0')
tensor([1106.0283], device='cuda:0')
tensor([772.8834], device='cuda:0')


Training consistently leads to worse results

In [ ]:
from utils import CNFFormula
from models.rbm import formulaRBM
import torch

with open('./test02.cnf', 'r') as f:
    formula_str = f.read()
formula = CNFFormula.from_dimacs(formula_str)

rbm_trained = formulaRBM(formula,
                         num_epochs=10000,
                         device="cuda")

print(rbm_trained.W_full)

tensor([[ 1.9978, -3.1180,  0.0000,  0.0000, -2.3297,  3.9816, -2.4516,  2.3680,
          1.0588,  2.0557,  1.2671, -4.1289,  2.4115],
        [-1.1680,  6.0914, -1.9978,  3.1180,  5.4193, -1.5878, -1.0259,  4.1523,
         -0.5033, -1.9225, -0.8347, -0.4400, -2.4606],
        [ 0.0000,  0.0000, -1.1680,  6.0914, -1.1188, -1.4281,  5.4664, -0.6311,
         -0.3739,  4.0862, -0.5167, -0.5561, -1.6324],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.6017,
          3.5612, -2.3678,  3.1999, -0.8163, -2.3878],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.3785,
         -0.1678, -0.7122, -0.6422, -0.3692,  4.1957]], device='cuda:0')


In [12]:
from models.rbm import clauseRBM

rbm_len2 = clauseRBM(2, F_s=-1.0, num_epochs=10000, lr=0.01, device='cuda')

print(rbm_len2.W)
print(rbm_len2.b)
print(rbm_len2.d)

Parameter containing:
tensor([[ 1.9978, -3.1180],
        [-1.1680,  6.0914]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-5.6222, -5.4554], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.9735, -0.0620], device='cuda:0', requires_grad=True)


In [57]:
import torch
import torch.nn as nn
import torch.optim as optim

class clauseRBM_symmetric(nn.Module):
    def __init__(self, n_visible = 2, n_hidden = 2, F_s=-1.0, num_epochs=100000, lr=0.1, verbose=False):
        super(clauseRBM_symmetric, self).__init__()
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.W = nn.Parameter(torch.randn(1, n_hidden) * 0.1)
        self.d = nn.Parameter(torch.zeros(1))
        self.b = nn.Parameter(torch.zeros(1))

        optimizer = optim.Adam(self.parameters(), lr=0.01)

        # Generate all possible binary vectors
        n_samples = 2 ** n_visible
        v_vectors = torch.tensor([ [int(x) for x in bin(i)[2:].zfill(n_visible)] for i in range(n_samples)], dtype=torch.float32)
        # print(v_vectors)
        # Target free energies
        F_targets = torch.zeros(n_samples)
        for i in range(n_samples):
            if v_vectors[i].sum() == 0:
                F_targets[i] = 0.0
            else:
                F_targets[i] = F_s
        # Training loop
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            F_v = self.free_energy(v_vectors)
            loss = torch.mean((F_v - F_targets) ** 2)
            loss.backward()
            optimizer.step()

            if (epoch+1) % 1000 == 0:
                print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    
    def free_energy(self, v):
        # v: batch_size x n_visible
        v_term = torch.sum(self.d * v, dim=1)
        # print(v_term)
        # print(self.W)
        W_expanded = self.W.expand(v.shape[1], self.n_hidden)
        # print(W_expanded)
        pre_activation = self.b + torch.matmul(v, W_expanded)
        # print(pre_activation)
        h_term = torch.sum(torch.log1p(torch.exp(pre_activation)), dim=1)
        F_v = -v_term - h_term
        # print(F_v)
        return F_v  # batch_size
    
rbm = clauseRBM_symmetric()
print(rbm.free_energy(torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])))
# print(rbm.W)

Epoch 1000, Loss: 0.06264595687389374
Epoch 2000, Loss: 0.06250002980232239
Epoch 3000, Loss: 0.0625000149011612
Epoch 4000, Loss: 0.0624999925494194
Epoch 5000, Loss: 0.0624999962747097
Epoch 6000, Loss: 0.062499985098838806
Epoch 7000, Loss: 0.0625000149011612
Epoch 8000, Loss: 0.0624999962747097
Epoch 9000, Loss: 0.0625000074505806
Epoch 10000, Loss: 0.06250017136335373
Epoch 11000, Loss: 0.0625000149011612
Epoch 12000, Loss: 0.0625
Epoch 13000, Loss: 0.062499985098838806
Epoch 14000, Loss: 0.0625000074505806
Epoch 15000, Loss: 0.0624999962747097
Epoch 16000, Loss: 0.0625000074505806
Epoch 17000, Loss: 0.0625026598572731
Epoch 18000, Loss: 0.0625
Epoch 19000, Loss: 0.0625
Epoch 20000, Loss: 0.0624999962747097
Epoch 21000, Loss: 0.0625
Epoch 22000, Loss: 0.0625
Epoch 23000, Loss: 0.0625
Epoch 24000, Loss: 0.0625
Epoch 25000, Loss: 0.0625
Epoch 26000, Loss: 0.0625
Epoch 27000, Loss: 0.0625
Epoch 28000, Loss: 0.0625
Epoch 29000, Loss: 0.0625
Epoch 30000, Loss: 0.0625
Epoch 31000, Loss: